Team Members:

SHILPA KUPPILI

DEEPSHIKHA MAHATO

KALYAN ROY

VENU KHARE

In [ ]:
!pip install transformers datasets torch


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained model and tokenizer
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained model and tokenizer
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Add a padding token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))


Embedding(50258, 768)

In [17]:
# Read data from a text file
with open('/content/Apple_Vision_Pro_Privacy_Overview.txt', 'r', encoding='utf-8') as file:
    custom_data = file.readlines()

# Tokenize the data
inputs = tokenizer(custom_data, return_tensors='pt', max_length=512, truncation=True, padding=True)

# Set the pad_token_id to the tokenizer's pad_token_id
model.config.pad_token_id = tokenizer.pad_token_id


In [18]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch

# Define training arguments
training_args = TrainingArguments(
    output_dir='/content/sample_data/Results',
    num_train_epochs=1,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
)

# Prepare the dataset for Trainer
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings['input_ids'])

dataset = CustomDataset(inputs)

# Create a DataCollator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# Train the model
trainer.train()

<ipython-input-18-a944ac7b9026>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss


TrainOutput(global_step=182, training_loss=3.1001482953082076, metrics={'train_runtime': 1380.813, 'train_samples_per_second': 0.263, 'train_steps_per_second': 0.132, 'total_flos': 35012621952000.0, 'train_loss': 3.1001482953082076, 'epoch': 1.0})

In [19]:
# Save the model and tokenizer
model.save_pretrained('/content/sample_data/custom_gpt')
tokenizer.save_pretrained('/content/sample_data/custom_gpt')

('/content/sample_data/custom_gpt/tokenizer_config.json',
 '/content/sample_data/custom_gpt/special_tokens_map.json',
 '/content/sample_data/custom_gpt/vocab.json',
 '/content/sample_data/custom_gpt/merges.txt',
 '/content/sample_data/custom_gpt/added_tokens.json')

In [21]:
# Load the fine-tuned model and tokenizer
custom_model = GPT2LMHeadModel.from_pretrained('/content/sample_data/custom_gpt')
custom_tokenizer = GPT2Tokenizer.from_pretrained('/content/sample_data/custom_gpt')

# Generate text
input_text = "Apple Vision Pro and visionOS minimize how much information developers"
input_ids = custom_tokenizer.encode(input_text, return_tensors='pt')
attention_mask = input_ids != custom_tokenizer.pad_token_id

# Ensure pad_token_id is set correctly
custom_model.config.pad_token_id = custom_tokenizer.pad_token_id

output = custom_model.generate(input_ids, attention_mask=attention_mask, max_length=50, pad_token_id=custom_model.config.eos_token_id)

# Decode the generated text
generated_text = custom_tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


Apple Vision Pro and visionOS minimize how much information developers can access.
, while processing information, can also access information about your surroundings.
, while processing information apps can access your surroundings.
, while processing information, can


Summary of building Custom GPT:

Load and Modify Pre-trained Model: Adjust the tokenizer and model to handle custom data, including adding padding tokens.

Prepare Custom Data: Read and tokenize custom training data from a text file.

Fine-Tune: Use the Trainer class to fine-tune the model on the custom data.

Save and Load: Save the fine-tuned model and tokenizer, and later load them for generating text.

Generate Text: Use the fine-tuned model to generate coherent text based on a given input prompt.

This process customizes the GPT-2 model to better fit specific needs or domains by fine-tuning it on relevant data.